In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

import time
import os
from datetime import datetime

In [3]:
provinces = ['BKK','Chiangmai','Khonkaen','Rayong','Saraburi','Surat']
provincesLatLng = dict()

In [4]:
for province in provinces:
    file = open(f'./{province}/position.txt','r').read().split('\n')
    temp = dict()
    temp['lat'] = file[0].split(' ')[2]
    temp['lng'] = file[1].split(' ')[2]
    provincesLatLng[province] =  temp
provincesLatLng

{'BKK': {'lat': '13.729984', 'lng': '100.536443'},
 'Chiangmai': {'lat': '18.840633', 'lng': '98.969661'},
 'Khonkaen': {'lat': '16.445329', 'lng': '102.835251'},
 'Rayong': {'lat': '12.671521', 'lng': '101.275875'},
 'Saraburi': {'lat': '14.685833', 'lng': '100.871996'},
 'Surat': {'lat': '9.126057', 'lng': '99.325355'}}

In [5]:
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)



====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been s

In [6]:
# 2020/07/01 01:00(utc+7) – 2021/06/30 22:00 => 2020/06/30/1800Z (utc+0) - 2021/06/30/1500Z (utc+0)
def formatTime(dt):
    temp = dt.split(' ')
    date0, time0 = temp[0] , temp[1]
    year,month,day = date0.split('-')
    date = ("/").join([day,month,year])
    hour = time0.split(':')[0]
    time = (":").join([str(int(hour)),'00','00'])
    date_time = (" ").join([date,time])
    return date_time

In [7]:
formatTime("2020-06-30 01:00 Local")

'30/06/2020 1:00:00'

In [10]:
#use time in utc+0
def scrapeWind(year,month,day,hour,province):
    lat, lng = provincesLatLng[province]['lat'], provincesLatLng[province]['lng']
    url = f'https://earth.nullschool.net/#{year}/{month:02d}/{day:02d}/{hour:02d}00Z/wind/isobaric/850hPa/anim=off/orthographic/loc={lng},{lat}'
    driver.get(url=url)
    #wait until
    element = WebDriverWait(driver,9999).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="spotlight-panel"]/div[2]/div')))
    earthButton = WebDriverWait(driver, 9999).until(EC.element_to_be_clickable((By.XPATH, '//*[@class="earth-bar"]/div/h1/button')))
    earthButton.click()
    elementTime = WebDriverWait(driver,9999).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="menu"]/div/div[2]/div[3]')))
    direction, speed = element.text.split(' @ ')
    direction = direction[:-1]
    date_time = formatTime(elementTime.text)
    print(date_time,speed,direction)
    # dummy web ไม่งั้น ตรงบรรทัด earthButton.click มันจะคลิกก่อนเว็บหน้าใหม่โหลดเสร็จ
    driver.get(url='https://google.com')
    return date_time,speed,direction

In [11]:
scrapeWind(2020,6,30,18,'BKK')

01/07/2020 1:00:00 42 260


('01/07/2020 1:00:00', '42', '260')

In [ ]:
nday = [0,31,28,31,30,31,30,31,31,30,31,30,31]

In [ ]:
temp = dict()
listDateTime = []
listDirection = []
listSpeed = []
date_time,speed,direction = scrapeWind(2020,6,30,18,'BKK')
listDateTime.append(date_time)
listSpeed.append(speed)
listDirection.append(direction)
date_time,speed,direction = scrapeWind(2021,6,30,6,'BKK')
listDateTime.append(date_time)
listSpeed.append(speed)
listDirection.append(direction)
print(listDateTime)
temp['date_time'] = listDateTime
temp['win speed'] = listSpeed
temp['wind dir'] = listDirection
print(temp)

In [ ]:
# 2021/07/01 01:00(utc+7) – 2021/07/01 22:00 => 2021/06/30/1800Z (utc+0) - 2021/07/01/1500Z (utc+0)
def scrapeAdditionalDay(province):
    df1 = pd.read_csv(f'./{province}_weather_wind_test.csv')
    temp2 = dict()
    listDateTime2 = []
    listDirection2 = []
    listSpeed2 = []
    # 01
    date_time,speed,direction= scrapeWind(2021,6,30,18,province)
    listDateTime2.append(date_time)
    listSpeed2.append(speed)
    listDirection2.append(direction)
    # 04
    date_time,speed,direction= scrapeWind(2021,6,30,21,province)
    listDateTime2.append(date_time)
    listSpeed2.append(speed)
    listDirection2.append(direction)
    # 07
    date_time,speed,direction= scrapeWind(2021,7,1,0,province)
    listDateTime2.append(date_time)
    listSpeed2.append(speed)
    listDirection2.append(direction)
    # 10
    date_time,speed,direction= scrapeWind(2021,7,1,3,province)
    listDateTime2.append(date_time)
    listSpeed2.append(speed)
    listDirection2.append(direction)
    # 13
    date_time,speed,direction= scrapeWind(2021,7,1,6,province)
    listDateTime2.append(date_time)
    listSpeed2.append(speed)
    listDirection2.append(direction)
    # 16
    date_time,speed,direction= scrapeWind(2021,7,1,9,province)
    listDateTime2.append(date_time)
    listSpeed2.append(speed)
    listDirection2.append(direction)
    # 19
    date_time,speed,direction= scrapeWind(2021,7,1,12,province)
    listDateTime2.append(date_time)
    listSpeed2.append(speed)
    listDirection2.append(direction)
    # 22
    date_time,speed,direction= scrapeWind(2021,7,1,15,province)
    listDateTime2.append(date_time)
    listSpeed2.append(speed)
    listDirection2.append(direction)
    temp2['date_time'] = listDateTime2
    temp2['win speed'] = listSpeed2
    temp2['wind dir'] = listDirection2
    df2 = pd.DataFrame(temp2)
    return pd.concat([df1,df2])

In [ ]:
# 2020/07/01 01:00(utc+7) – 2021/06/30 22:00 => 2020/06/30/1800Z (utc+0) - 2021/06/30/1500Z (utc+0)
def getWindData(province):
    temp = dict()
    listDateTime = []
    listDirection = []
    listSpeed = []
    y = 2020
    date_time,speed,direction = scrapeWind(2020,6,30,18,province)
    listDateTime.append(date_time)
    listSpeed.append(speed)
    listDirection.append(direction)
    date_time,speed,direction = scrapeWind(2020,6,30,21,province)
    listDateTime.append(date_time)
    listSpeed.append(speed)
    listDirection.append(direction)
    for m in range(7,13):
        for d in range(1,32):
            if(d > nday[m]):
                continue
            for h in range(0,24,3):
                date_time,speed,direction = scrapeWind(y,m,d,h,province)
                listDateTime.append(date_time)
                listSpeed.append(speed)
                listDirection.append(direction)
    y = 2021
    for m in range(1,7):
        for d in range(1,32):
            if(d > nday[m]):
                continue
            for h in range(0,24,3):
                date_time,speed,direction = scrapeWind(y,m,d,h,province)
                listDateTime.append(date_time)
                listSpeed.append(speed)
                listDirection.append(direction)
                if(m==6 and d== 30 and h == 15):
                    break
    temp['date_time'] = listDateTime
    temp['win speed'] = listSpeed
    temp['wind dir'] = listDirection
    return temp

In [ ]:
windBKK = getWindData('BKK')

In [ ]:
dfBKK = pd.DataFrame(windBKK)
dfBKK.to_csv('bkk_weather_wind_test.csv', index=False)

In [ ]:
dfBKKNew = scrapeAdditionalDay('BKK')
dfBKKNew.to_csv('BKK_weather_wind_test2.csv', index=False)

In [ ]:
windChiangmai = getWindData('Chiangmai')

In [ ]:
dfChiangmai = pd.DataFrame(windChiangmai)
dfChiangmai.to_csv('chiangmai_weather_wind_test.csv', index=False)

In [ ]:
dfChiangmaiNew = scrapeAdditionalDay('Chiangmai')
dfChiangmaiNew.to_csv('Chiangmai_weather_wind_test2.csv', index=False)

In [ ]:
windKhonkaen = getWindData('Khonkaen')

In [ ]:
dfKhonkaen = pd.DataFrame(windKhonkaen)
dfKhonkaen.to_csv('khonkaen_weather_wind_test.csv', index=False)

In [ ]:
dfKhonkaenNew = scrapeAdditionalDay('Khonkaen')
dfKhonkaenNew.to_csv('Khonkaen_weather_wind_test2.csv', index=False)

In [ ]:
windRayong = getWindData('Rayong')

In [ ]:
dfRayong = pd.DataFrame(windRayong)
dfRayong.to_csv('rayong_weather_wind_test.csv', index=False)

In [ ]:
dfRayongNew = scrapeAdditionalDay('Rayong')
dfRayongNew.to_csv('Rayong_weather_wind_test2.csv', index=False)

In [ ]:
windSaraburi = getWindData('Saraburi')

In [ ]:
dfSaraburi = pd.DataFrame(windSaraburi)
dfSaraburi.to_csv('saraburi_weather_wind_test.csv', index=False)

In [ ]:
dfSaraburiNew = scrapeAdditionalDay('Saraburi')
dfSaraburiNew.to_csv('Saraburi_weather_wind_test2.csv', index=False)

In [ ]:
windSurat = getWindData('Surat')

In [ ]:
dfSurat = pd.DataFrame(windSurat)
dfSurat.to_csv('surat_weather_wind_test.csv', index=False)

In [ ]:
dfSuratNew = scrapeAdditionalDay('Surat')
dfSuratNew.to_csv('Surat_weather_wind_test2.csv', index=False)

In [ ]:
driver.quit()